# From image idea to Kibana dashboard using AI

This notebook is based on the article [From image idea to Kibana dashboard using AI](https://www.elastic.co/search-labs/blog/from-image-idea-to-kibana-dashboard-using-ai). With the following code, we can generate a Kibana dashboard from an image.

Note: This notebook was done to be executed in Google Colab.

## Install dependencies

In [ ]:
%pip install elasticsearch pydantic langchain langchain-openai -q

In [115]:
import requests, time, os, base64, json, uuid, urllib.parse
from IPython.display import Image, display
from getpass import getpass
from typing import Any, Dict, List, Literal, Optional

from google.colab import files

from elasticsearch import Elasticsearch
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

## Defining the dashboard schema

In [105]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Enter your Elasticsearch API key: ")
os.environ["ELASTICSEARCH_URL"] = getpass("Enter your Elasticsearch URL: ")
os.environ["KIBANA_URL"] = getpass("Enter your Kibana URL: ")

## Defining the dashboard schema

In [106]:
class Visualization(BaseModel):
    title: str = Field(description="The dashboard title")
    type: List[Literal["pie", "bar", "metric"]]
    field: Optional[str] = Field(
        description="The field that this visualization use based on the provided mappings"
    )


class Dashboard(BaseModel):
    title: str = Field(description="The dashboard title")
    visualizations: List[Visualization]

## Loading the json templates

There are 3 templates for each visualization type:
- pie
- bar
- metric

The templates are in the following format:
- insBar.json
- insPie.json
- insMetric.json

You can find the templates in the `/templates` folder.

In [ ]:
uploaded = files.upload()

template_dir = "templates"
os.makedirs(template_dir, exist_ok=True)

for filename in uploaded.keys():
    with open(os.path.join(template_dir, filename), "wb") as f:
        f.write(uploaded[filename])

templates = {}
for vis_type in ["pie", "bar", "metric"]:
    template_file = os.path.join(template_dir, f"lns{vis_type.capitalize()}.json")

    with open(template_file, "r") as f:
        templates[vis_type] = json.load(f)

    if not templates:
        print("No templates found")
        break

    print(f"Loaded {len(templates)} templates")

## Defining the dashboard schema

Retrieve index mappings for the index that the dashboard is based on.

In [94]:
INDEX_NAME = "kibana_sample_data_logs"

es_client = Elasticsearch(
    [os.getenv("ELASTICSEARCH_URL")],
    api_key=os.getenv("ELASTICSEARCH_API_KEY"),
)

result = es_client.indices.get_mapping(index=INDEX_NAME)
index_mappings = result[list(result.keys())[0]]["mappings"]["properties"]

## Loading image 
You can load an image from your local machine and use it to generate a dashboard. In the `/imgs` folder, you can find images to use as examples.

In [30]:
uploaded = files.upload()

IMAGE_PATH = next(iter(uploaded.keys()))
image_base64 = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("utf-8")

In [ ]:
prompt = f"""
    You are an expert in analyzing Kibana dashboards from images for the version 9.0.0 of Kibana.

    You will be given a dashboard image and a Elasticsearch index mappings.

    Below is the index mappings for the index that the dashboard is based on. Use this to help you understand the data and the fields that are available.

    Index Mappings:
    {index_mappings}

    Only include the fields that are relevant for each visualization, based on what is visible in the image.
    """

message = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {
                "type": "image",
                "source_type": "base64",
                "data": image_base64,
                "mime_type": "image/png",
            },
        ],
    }
]


try:
    llm = init_chat_model("gpt-4.1-mini")
    llm = llm.with_structured_output(Dashboard)
    dashboard_values = llm.invoke(message)

    print("Dashboard values generated by the LLM successfully")
    print(dashboard_values)
except Exception as e:
    print(f"Failed to analyze image and match fields: {str(e)}")

Filling the template with the values generated by the LLM:

In [116]:
def fill_template_with_analysis(
    template: Dict[str, Any],
    visualization: Visualization,
    grid_data: Dict[str, Any],
):
    template_str = json.dumps(template)
    replacements = {
        "{title}": visualization.title,
        "{x}": grid_data["x"],
        "{y}": grid_data["y"],
    }

    if visualization.field:
        replacements["{field}"] = visualization.field

    for placeholder, value in replacements.items():
        template_str = template_str.replace(placeholder, str(value))

    return json.loads(template_str)

In [ ]:
panels = []
grid_data = [
    {
        "x": 0,
        "y": 0,
    },
    {
        "x": 12,
        "y": 0,
    },
    {
        "x": 6,
        "y": 12,
    },
]

i = 0

for vis in dashboard_values.visualizations:
    for vis_type in vis.type:
        template = templates.get(vis_type, templates.get("bar", {}))
        filled_panel = fill_template_with_analysis(template, vis, grid_data[i])
        panels.append(filled_panel)
        i += 1

## Generate the dashboard

Here is called the API `/api/generate-dashboard`. The templates with the values generated by the LLM are sent to the API.

In [ ]:
try:
    dashboard_id = str(uuid.uuid4())

    # post request to create the dashboard endpoint
    url = f"{os.getenv('KIBANA_URL')}/api/dashboards/dashboard/{dashboard_id}"

    dashboard_config = {
        "attributes": {
            "title": dashboard_values.title,
            "description": "Generated by AI",
            "timeRestore": True,
            "panels": panels,  # Visualizations with the values generated by the LLM
            "timeFrom": "now-7d/d",
            "timeTo": "now",
        },
    }

    headers = {
        "Content-Type": "application/json",
        "kbn-xsrf": "true",
        "Authorization": f"ApiKey {os.getenv('ELASTICSEARCH_API_KEY')}",
    }

    requests.post(
        url,
        headers=headers,
        json=dashboard_config,
    )

    # Url to the generated dashboard
    dashboard_url = f"{os.getenv('KIBANA_URL')}/app/dashboards#/view/{dashboard_id}"

    print("Dashboard URL: ", dashboard_url)
    print("Dashboard ID: ", dashboard_id)

except Exception as e:
    print(f"Failed to create dashboard: {str(e)}")

## Generating a dashboard image from Kibana

In [ ]:
screenshot_height = 680
screenshot_width = 1418

job_params = (
    f"(browserTimezone:America/Panama,"
    f"layout:(dimensions:(height:{screenshot_height},width:{screenshot_width}),id:preserve_layout),"
    f"locatorParams:(id:DASHBOARD_APP_LOCATOR,params:(dashboardId:'{dashboard_id}',"
    f"preserveSavedFilters:!t,"
    f"timeRange:(from:now-7d/d,to:now),"
    f"useHash:!f,viewMode:view)),"
    f"objectType:dashboard,"
    f"title:'Web Traffic Overview',"
    f"version:'9.0.0')"
)

# Creating job report request
job_params_str = urllib.parse.quote(job_params)

url = (
    f"{os.getenv('KIBANA_URL')}/api/reporting/generate/pngV2?jobParams={job_params_str}"
)
headers = {
    "kbn-xsrf": "true",
    "Authorization": f"ApiKey {os.getenv('ELASTICSEARCH_API_KEY')}",
}

r = requests.post(url, headers=headers)

if r.status_code != 200:
    raise Exception("Failed to start report job")

job_url = f"{os.getenv('KIBANA_URL')}{r.json()['path']}"

# Wait for job to complete
while True:
    resp = requests.get(job_url, headers=headers)
    ctype = resp.headers.get("Content-Type", "")

    if "image/png" in ctype:
        with open("dashboard.png", "wb") as f:
            f.write(resp.content)

        print("PNG downloaded as dashboard.png")
        display(Image("dashboard.png"))
        break
    else:
        print("Waiting for job to complete...")

    time.sleep(5)